In [ ]:
from __future__ import print_function
from google.colab import files 

import io
import pandas as pd


  
#Code to upload the dataset from your computer as a dataframe
df=pd.read_csv("/content/preprocessed_data.csv")


In [ ]:
df.head(10)

,Unnamed: 0,Unnamed: 0.1,product,desc,product_id
0,0,0,Strap top,solid jersey basic basic everyday woman should...,1
1,1,1,Strap top (1),stripe jersey basic basic everyday woman shoul...,2
2,2,2,OP T-shirt (Idro),solid clean lingerie woman lingerie visible mi...,3
3,3,3,20 den 1p Stockings,solid tights basic tigh sock nightwear woman w...,4
4,4,4,Shape Up 30 den 1p Tights,solid tights basic tigh sock nightwear woman a...,5
5,5,5,Support 40 den 1p Tights,solid tights basic tigh sock nightwear woman e...,6
6,6,6,200 den 1p Tights,solid tights basic tigh sock nightwear woman m...,7
7,7,7,SWEATSHIRT OC,pattern basic baby complement essential baby p...,8
8,8,8,Alice BANDEAU 2-p,stripesolid casual lingerie woman lingerie top...,9
9,9,9,Frugan longsleeve,solid jersey basic basic everyday woman neckli...,10


In [ ]:
#Perform Word Embedding
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
!wget https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g -O GoogleGoogleNews-vectors-negative300.bin

--2022-10-21 14:12:39--  https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g
Resolving drive.google.com (drive.google.com)... 74.125.142.101, 74.125.142.113, 74.125.142.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘GoogleGoogleNews-vectors-negative300.bin’

GoogleGoogleNews-ve     [ <=>                ]  66.61K  --.-KB/s    in 0.03s   

2022-10-21 14:12:40 (2.32 MB/s) - ‘GoogleGoogleNews-vectors-negative300.bin’ saved [68207]



In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: glove.6B.100d.txt       


In [ ]:

import os
# Global parameters
#root folder
root_folder='.'
data_folder_name='data'
glove_filename='glove.6B.100d.txt'


# Variable for data directory
DATA_PATH = os.path.abspath(os.path.join(root_folder, data_folder_name))
glove_path = "/content/glove.6B.100d.txt"


In [ ]:
# glove2word2vec saves the Glove embeddings in the word2vec format 
from gensim.scripts.glove2word2vec import glove2word2vec
word2vec_output_file = glove_filename+'.word2vec'
glove2word2vec(glove_path, word2vec_output_file)

(400000, 100)

In [ ]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
word2vec_output_file = glove_filename+'.word2vec'
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)


In [ ]:
d_arr = df['desc'].to_numpy()
print(d_arr)


['solid jersey basic basic everyday woman shoulder top jersey strap narrow'
 'stripe jersey basic basic everyday woman shoulder top jersey strap narrow'
 'solid clean lingerie woman lingerie visible microfibre back padded comfort good support without bra hookandeye cup fastening shoulder seam provide bust greater lightly moulded tshirt underwired shape strap narrow adjustable'
 ...
 'solidmelangestructure jersey basic light men underwear elasticated drawstring pocket waist seam short cotton side piqué discreet'
 'solid jersey basic light men underwear sturdy roundnecked tshirt cotton jersey'
 'melangestripesolid jerseyjersey basic light men underwear sturdy roundnecked tshirt cotton jersey']


In [ ]:
#Maximum length of the vectors
final_vector =  {}
m = 0
for i in range (0,10000):

  word_vec = []
  for w in d_arr[i].split(" "):
    try:
      word_vec.append(sum(model.get_vector(w))/100)
    except:
      pass
  if m < len(word_vec):
    m = len(word_vec)
  final_vector[df['product'][i]] = word_vec
print(m)

67


In [ ]:
#Zero-padding of vectors
for key in final_vector.keys():
  if len(final_vector[key]) < m:
    r = m - len(final_vector[key])
    l = [0 for i in range(r)]
    final_vector[key].extend(l)

In [ ]:
# make the matrix with the words
words = final_vector.keys()
X = []
for w in words:
    X.append(final_vector[w])

from sklearn import cluster
from sklearn import metrics
kmeans = cluster.KMeans(n_clusters=500)
kmeans.fit(X)
print(kmeans)
 
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
 
print ("Cluster id labels for input data")
print (labels)
print ("Centroids data")
print (centroids)

KMeans(n_clusters=500)
Cluster id labels for input data
[455 455 385 ...  60  58  72]
Centroids data
[[ 7.21139180e-03  4.40157635e-02 -1.06072633e-02 ... -8.47032947e-22
   0.00000000e+00 -2.11758237e-22]
 [ 2.70615277e-02  5.01252282e-02  6.03179212e-02 ... -8.47032947e-22
   0.00000000e+00 -2.11758237e-22]
 [ 1.38760970e-02 -2.06191902e-02  2.62018844e-02 ... -8.47032947e-22
   8.47032947e-22  0.00000000e+00]
 ...
 [ 2.20414948e-02 -4.20187521e-02  7.27292312e-02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.14648466e-02  2.78259793e-02  4.25330905e-02 ...  0.00000000e+00
   0.00000000e+00 -2.11758237e-22]
 [ 1.08234348e-02 -6.10507708e-02 -3.85645672e-02 ... -8.47032947e-22
   8.47032947e-22  0.00000000e+00]]


In [ ]:
import numpy as np

print(np.array(X).shape)


(10000, 67)


In [ ]:
#Find the cluster with least distance with the query vector
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
query ="Strap top"
v = np.array([final_vector[query]])
dist = 10000
for i in range(len(centroids)):
  d = cosine_similarity(v,[centroids[i]])
  if dist > d:
    
    dist = d
    label = labels[i]





In [ ]:
label

369

In [ ]:
#Getting the items present in the cluster.
ref = { i:X[i] for i in list(np.where(labels ==label)[0]) }
reff=[]
reff_key =[]
reff_key = list(ref.keys())
reff = list(ref.values())

In [ ]:
#distance between the query and the items present within the cluster
dist_list=[]
index_id =[]
v = np.array([final_vector[query]])
for i in range(len(reff)):
  d = cosine_similarity(v,[reff[i]])
  dist_list.append(list(d))
  index_id.append(reff_key[i])
  

In [ ]:
length = len(dist_list)

In [ ]:
length

19

In [ ]:
index_id

[25,
 148,
 995,
 1597,
 1648,
 1728,
 1729,
 2476,
 2827,
 3049,
 3319,
 4047,
 4382,
 4938,
 5990,
 6176,
 7439,
 8710,
 9296]

In [ ]:
print(dist_list)

[[array([0.18663209])], [array([0.08893923])], [array([0.20763201])], [array([0.42633919])], [array([0.17032905])], [array([0.03988385])], [array([0.05892014])], [array([0.01521287])], [array([-0.03741408])], [array([-0.05125366])], [array([0.15209587])], [array([0.1363246])], [array([0.07749531])], [array([0.23781763])], [array([0.31858244])], [array([0.0621184])], [array([0.13549363])], [array([0.31858244])], [array([0.22096158])]]


In [ ]:
#Dictionary for index and respective difference of distance.
dictionary_list ={}
for i in range(len(dist_list)):
  dictionary_list[index_id[i]] =  dist_list[i]



In [ ]:
dictionary_list

{25: [array([0.18663209])],
 148: [array([0.08893923])],
 995: [array([0.20763201])],
 1597: [array([0.42633919])],
 1648: [array([0.17032905])],
 1728: [array([0.03988385])],
 1729: [array([0.05892014])],
 2476: [array([0.01521287])],
 2827: [array([-0.03741408])],
 3049: [array([-0.05125366])],
 3319: [array([0.15209587])],
 4047: [array([0.1363246])],
 4382: [array([0.07749531])],
 4938: [array([0.23781763])],
 5990: [array([0.31858244])],
 6176: [array([0.0621184])],
 7439: [array([0.13549363])],
 8710: [array([0.31858244])],
 9296: [array([0.22096158])]}

In [ ]:
#Sorting in ascending order of distance value
dist_list.sort

<function list.sort(*, key=None, reverse=False)>

In [ ]:
#Top 5 vectors with least difference wrt distance
min_five_dist =[]
for i in range(0,5):
  min_five_dist.append(dist_list[i])
  

In [ ]:
min_five_dist

[[array([0.18663209])],
 [array([0.08893923])],
 [array([0.20763201])],
 [array([0.42633919])],
 [array([0.17032905])]]

In [ ]:
#Product id of the vectors
top_five_index = [key for key,val in dictionary_list.items() if val in min_five_dist[0:len(min_five_dist)]]
top_five_index

[25, 148, 995, 1597, 1648]

In [ ]:
#Product names based on similarity
product_list =[]
for i in range(len(top_five_index)):
  product_list.append(df[df['product_id'] == top_five_index[i]]['product'].values[0])

In [ ]:
print(product_list)

['Nouvelle 1p Stay Up', 'Fix it tote bag', 'George Beanie', 'Priscilla price t-shirt', 'ALBIN basic rib l/s 2p SB']
